## Data preprocessing

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

In [15]:
# Set random seed for reproducibility
np.random.seed(42)

# Load merged data
data = pd.read_csv('merged_cab_weather_data.csv')


In [16]:
# Convert 'timestamp' to datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.head()

timestamp source_address destination_address  number_of_cabs  \
0 2022-12-01 12:47:24    Koramangala         Indiranagar              34   
1 2021-01-28 08:40:56    Indiranagar             MG Road              24   
2 2021-10-28 21:10:11     HSR Layout         Indiranagar              26   
3 2021-09-02 04:37:16     HSR Layout         Indiranagar              21   
4 2023-03-17 16:47:17    Indiranagar             MG Road              42   

   base_price  adjusted_price day_of_week  is_working_day  is_holiday  month  \
0      256.97          334.06    Thursday               1           0     12   
1      262.71          512.28    Thursday               1           0      1   
2      261.09          339.42    Thursday               1           0     10   
3      214.01          278.21    Thursday               1           0      9   
4      206.46          268.40      Friday               1           0      3   

  weather_condition  
0             Rainy  
1             Rainy  
2             Rainy  
3             Foggy  
4             Rainy

In [17]:
# Extract hour, day of the week, and month from the timestamp
data['hour'] = data['timestamp'].dt.hour
data['day_of_week'] = data['timestamp'].dt.dayofweek
data['month'] = data['timestamp'].dt.month
data.head()

timestamp source_address destination_address  number_of_cabs  \
0 2022-12-01 12:47:24    Koramangala         Indiranagar              34   
1 2021-01-28 08:40:56    Indiranagar             MG Road              24   
2 2021-10-28 21:10:11     HSR Layout         Indiranagar              26   
3 2021-09-02 04:37:16     HSR Layout         Indiranagar              21   
4 2023-03-17 16:47:17    Indiranagar             MG Road              42   

   base_price  adjusted_price  day_of_week  is_working_day  is_holiday  month  \
0      256.97          334.06            3               1           0     12   
1      262.71          512.28            3               1           0      1   
2      261.09          339.42            3               1           0     10   
3      214.01          278.21            3               1           0      9   
4      206.46          268.40            4               1           0      3   

  weather_condition  hour  
0             Rainy    12  
1             Rainy     8  
2             Rainy    21  
3             Foggy     4  
4             Rainy    16

In [18]:
# Handle missing values using forward fill
data.ffill(inplace=True)

In [19]:
# Step 2: Feature Engineering (Add new features here)
# Adding Rush Hour feature
data['is_rush_hour'] = data['hour'].apply(lambda x: 1 if (7 <= x <= 9) or (17 <= x <= 19) else 0)

In [20]:
# Adding Weekend feature (if not already present)
data['is_weekend'] = data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

In [21]:
data.head()

timestamp source_address destination_address  number_of_cabs  \
0 2022-12-01 12:47:24    Koramangala         Indiranagar              34   
1 2021-01-28 08:40:56    Indiranagar             MG Road              24   
2 2021-10-28 21:10:11     HSR Layout         Indiranagar              26   
3 2021-09-02 04:37:16     HSR Layout         Indiranagar              21   
4 2023-03-17 16:47:17    Indiranagar             MG Road              42   

   base_price  adjusted_price  day_of_week  is_working_day  is_holiday  month  \
0      256.97          334.06            3               1           0     12   
1      262.71          512.28            3               1           0      1   
2      261.09          339.42            3               1           0     10   
3      214.01          278.21            3               1           0      9   
4      206.46          268.40            4               1           0      3   

  weather_condition  hour  is_rush_hour  is_weekend  
0             Rainy    12             0           0  
1             Rainy     8             1           0  
2             Rainy    21             0           0  
3             Foggy     4             0           0  
4             Rainy    16             0           0

In [22]:
# Handling missing values and encoding categorical variables
data = pd.get_dummies(data, columns=['weather_condition'], drop_first=True)
data.head()

timestamp source_address destination_address  number_of_cabs  \
0 2022-12-01 12:47:24    Koramangala         Indiranagar              34   
1 2021-01-28 08:40:56    Indiranagar             MG Road              24   
2 2021-10-28 21:10:11     HSR Layout         Indiranagar              26   
3 2021-09-02 04:37:16     HSR Layout         Indiranagar              21   
4 2023-03-17 16:47:17    Indiranagar             MG Road              42   

   base_price  adjusted_price  day_of_week  is_working_day  is_holiday  month  \
0      256.97          334.06            3               1           0     12   
1      262.71          512.28            3               1           0      1   
2      261.09          339.42            3               1           0     10   
3      214.01          278.21            3               1           0      9   
4      206.46          268.40            4               1           0      3   

   hour  is_rush_hour  is_weekend  weather_condition_Foggy  \
0    12             0           0                    False   
1     8             1           0                    False   
2    21             0           0                    False   
3     4             0           0                     True   
4    16             0           0                    False   

   weather_condition_Rainy  weather_condition_Sunny  
0                     True                    False  
1                     True                    False  
2                     True                    False  
3                    False                    False  
4                     True                    False

In [23]:
# Adding Interaction Terms for Weather and Time
data['weather_hour_interaction'] = data['hour'] * (data['weather_condition_Sunny'] + data['weather_condition_Rainy'] + data['weather_condition_Foggy'])

In [18]:
data.head()

timestamp source_address destination_address  number_of_cabs  \
0 2022-12-01 12:47:24    Koramangala         Indiranagar              34   
1 2021-01-28 08:40:56    Indiranagar             MG Road              24   
2 2021-10-28 21:10:11     HSR Layout         Indiranagar              26   
3 2021-09-02 04:37:16     HSR Layout         Indiranagar              21   
4 2023-03-17 16:47:17    Indiranagar             MG Road              42   

   base_price  adjusted_price  day_of_week  is_working_day  is_holiday  month  \
0      256.97          334.06            3               1           0     12   
1      262.71          512.28            3               1           0      1   
2      261.09          339.42            3               1           0     10   
3      214.01          278.21            3               1           0      9   
4      206.46          268.40            4               1           0      3   

   hour  is_rush_hour  is_weekend  weather_condition_Foggy  \
0    12             0           0                    False   
1     8             1           0                    False   
2    21             0           0                    False   
3     4             0           0                     True   
4    16             0           0                    False   

   weather_condition_Rainy  weather_condition_Sunny  weather_hour_interaction  
0                     True                    False                        12  
1                     True                    False                         8  
2                     True                    False                        21  
3                    False                    False                         4  
4                     True                    False                        16

In [24]:
# Feature Engineering
# Lag Features
data['lag_1'] = data['number_of_cabs'].shift(1)
data['lag_2'] = data['number_of_cabs'].shift(2)
data.head()

timestamp source_address destination_address  number_of_cabs  \
0 2022-12-01 12:47:24    Koramangala         Indiranagar              34   
1 2021-01-28 08:40:56    Indiranagar             MG Road              24   
2 2021-10-28 21:10:11     HSR Layout         Indiranagar              26   
3 2021-09-02 04:37:16     HSR Layout         Indiranagar              21   
4 2023-03-17 16:47:17    Indiranagar             MG Road              42   

   base_price  adjusted_price  day_of_week  is_working_day  is_holiday  month  \
0      256.97          334.06            3               1           0     12   
1      262.71          512.28            3               1           0      1   
2      261.09          339.42            3               1           0     10   
3      214.01          278.21            3               1           0      9   
4      206.46          268.40            4               1           0      3   

   hour  is_rush_hour  is_weekend  weather_condition_Foggy  \
0    12             0           0                    False   
1     8             1           0                    False   
2    21             0           0                    False   
3     4             0           0                     True   
4    16             0           0                    False   

   weather_condition_Rainy  weather_condition_Sunny  weather_hour_interaction  \
0                     True                    False                        12   
1                     True                    False                         8   
2                     True                    False                        21   
3                    False                    False                         4   
4                     True                    False                        16   

   lag_1  lag_2  
0    NaN    NaN  
1   34.0    NaN  
2   24.0   34.0  
3   26.0   24.0  
4   21.0   26.0

In [25]:
# Adding Rolling Mean and Lag features for number of cabs
data['rolling_mean_cabs'] = data['number_of_cabs'].rolling(window=3).mean().fillna(0)
data['lag_cabs_3'] = data['number_of_cabs'].shift(3).fillna(0)

In [21]:
data.head()

timestamp source_address destination_address  number_of_cabs  \
0 2022-12-01 12:47:24    Koramangala         Indiranagar              34   
1 2021-01-28 08:40:56    Indiranagar             MG Road              24   
2 2021-10-28 21:10:11     HSR Layout         Indiranagar              26   
3 2021-09-02 04:37:16     HSR Layout         Indiranagar              21   
4 2023-03-17 16:47:17    Indiranagar             MG Road              42   

   base_price  adjusted_price  day_of_week  is_working_day  is_holiday  month  \
0      256.97          334.06            3               1           0     12   
1      262.71          512.28            3               1           0      1   
2      261.09          339.42            3               1           0     10   
3      214.01          278.21            3               1           0      9   
4      206.46          268.40            4               1           0      3   

   ...  is_rush_hour  is_weekend  weather_condition_Foggy  \
0  ...             0           0                    False   
1  ...             1           0                    False   
2  ...             0           0                    False   
3  ...             0           0                     True   
4  ...             0           0                    False   

   weather_condition_Rainy  weather_condition_Sunny  weather_hour_interaction  \
0                     True                    False                        12   
1                     True                    False                         8   
2                     True                    False                        21   
3                    False                    False                         4   
4                     True                    False                        16   

   lag_1  lag_2  rolling_mean_cabs  lag_cabs_3  
0    NaN    NaN           0.000000         0.0  
1   34.0    NaN           0.000000         0.0  
2   24.0   34.0          28.000000         0.0  
3   26.0   24.0          23.666667        34.0  
4   21.0   26.0          29.666667        24.0  

[5 rows x 21 columns]

In [26]:
# Filling NaN values created by lag features
data.fillna(0, inplace=True)
data.head()

timestamp source_address destination_address  number_of_cabs  \
0 2022-12-01 12:47:24    Koramangala         Indiranagar              34   
1 2021-01-28 08:40:56    Indiranagar             MG Road              24   
2 2021-10-28 21:10:11     HSR Layout         Indiranagar              26   
3 2021-09-02 04:37:16     HSR Layout         Indiranagar              21   
4 2023-03-17 16:47:17    Indiranagar             MG Road              42   

   base_price  adjusted_price  day_of_week  is_working_day  is_holiday  month  \
0      256.97          334.06            3               1           0     12   
1      262.71          512.28            3               1           0      1   
2      261.09          339.42            3               1           0     10   
3      214.01          278.21            3               1           0      9   
4      206.46          268.40            4               1           0      3   

   ...  is_rush_hour  is_weekend  weather_condition_Foggy  \
0  ...             0           0                    False   
1  ...             1           0                    False   
2  ...             0           0                    False   
3  ...             0           0                     True   
4  ...             0           0                    False   

   weather_condition_Rainy  weather_condition_Sunny  weather_hour_interaction  \
0                     True                    False                        12   
1                     True                    False                         8   
2                     True                    False                        21   
3                    False                    False                         4   
4                     True                    False                        16   

   lag_1  lag_2  rolling_mean_cabs  lag_cabs_3  
0    0.0    0.0           0.000000         0.0  
1   34.0    0.0           0.000000         0.0  
2   24.0   34.0          28.000000         0.0  
3   26.0   24.0          23.666667        34.0  
4   21.0   26.0          29.666667        24.0  

[5 rows x 21 columns]

In [27]:
# Interaction Features
data['hour_weather_Rainy'] = data['hour'] * data['weather_condition_Rainy']
# Interaction feature for Sunny weather
data['hour_weather_Sunny'] = data['hour'] * data['weather_condition_Sunny']
# Interaction feature for Foggy weather
data['hour_weather_Foggy'] = data['hour'] * data['weather_condition_Foggy']
data.head()

timestamp source_address destination_address  number_of_cabs  \
0 2022-12-01 12:47:24    Koramangala         Indiranagar              34   
1 2021-01-28 08:40:56    Indiranagar             MG Road              24   
2 2021-10-28 21:10:11     HSR Layout         Indiranagar              26   
3 2021-09-02 04:37:16     HSR Layout         Indiranagar              21   
4 2023-03-17 16:47:17    Indiranagar             MG Road              42   

   base_price  adjusted_price  day_of_week  is_working_day  is_holiday  month  \
0      256.97          334.06            3               1           0     12   
1      262.71          512.28            3               1           0      1   
2      261.09          339.42            3               1           0     10   
3      214.01          278.21            3               1           0      9   
4      206.46          268.40            4               1           0      3   

   ...  weather_condition_Rainy  weather_condition_Sunny  \
0  ...                     True                    False   
1  ...                     True                    False   
2  ...                     True                    False   
3  ...                    False                    False   
4  ...                     True                    False   

   weather_hour_interaction  lag_1  lag_2  rolling_mean_cabs  lag_cabs_3  \
0                        12    0.0    0.0           0.000000         0.0   
1                         8   34.0    0.0           0.000000         0.0   
2                        21   24.0   34.0          28.000000         0.0   
3                         4   26.0   24.0          23.666667        34.0   
4                        16   21.0   26.0          29.666667        24.0   

   hour_weather_Rainy  hour_weather_Sunny  hour_weather_Foggy  
0                  12                   0                   0  
1                   8                   0                   0  
2                  21                   0                   0  
3                   0                   0                   4  
4                  16                   0                   0  

[5 rows x 24 columns]

In [28]:
#verification for prev step
print(data[['hour', 'weather_condition_Rainy', 'hour_weather_Rainy', 'hour_weather_Sunny', 'hour_weather_Foggy']].head())


   hour  weather_condition_Rainy  hour_weather_Rainy  hour_weather_Sunny  \
0    12                     True                  12                   0   
1     8                     True                   8                   0   
2    21                     True                  21                   0   
3     4                    False                   0                   0   
4    16                     True                  16                   0   

   hour_weather_Foggy  
0                   0  
1                   0  
2                   0  
3                   4  
4                   0  


In [29]:
data.head()

timestamp source_address destination_address  number_of_cabs  \
0 2022-12-01 12:47:24    Koramangala         Indiranagar              34   
1 2021-01-28 08:40:56    Indiranagar             MG Road              24   
2 2021-10-28 21:10:11     HSR Layout         Indiranagar              26   
3 2021-09-02 04:37:16     HSR Layout         Indiranagar              21   
4 2023-03-17 16:47:17    Indiranagar             MG Road              42   

   base_price  adjusted_price  day_of_week  is_working_day  is_holiday  month  \
0      256.97          334.06            3               1           0     12   
1      262.71          512.28            3               1           0      1   
2      261.09          339.42            3               1           0     10   
3      214.01          278.21            3               1           0      9   
4      206.46          268.40            4               1           0      3   

   ...  weather_condition_Rainy  weather_condition_Sunny  \
0  ...                     True                    False   
1  ...                     True                    False   
2  ...                     True                    False   
3  ...                    False                    False   
4  ...                     True                    False   

   weather_hour_interaction  lag_1  lag_2  rolling_mean_cabs  lag_cabs_3  \
0                        12    0.0    0.0           0.000000         0.0   
1                         8   34.0    0.0           0.000000         0.0   
2                        21   24.0   34.0          28.000000         0.0   
3                         4   26.0   24.0          23.666667        34.0   
4                        16   21.0   26.0          29.666667        24.0   

   hour_weather_Rainy  hour_weather_Sunny  hour_weather_Foggy  
0                  12                   0                   0  
1                   8                   0                   0  
2                  21                   0                   0  
3                   0                   0                   4  
4                  16                   0                   0  

[5 rows x 24 columns]

In [30]:
# One-hot encode categorical variables
data = pd.get_dummies(data, columns=['source_address', 'destination_address'])

In [26]:
# Step 3: Encoding Categorical Variables
# One-hot encode categorical variables
data = pd.get_dummies(data, columns=['source_address', 'destination_address'])

In [31]:
data.head()

timestamp  number_of_cabs  base_price  adjusted_price  \
0 2022-12-01 12:47:24              34      256.97          334.06   
1 2021-01-28 08:40:56              24      262.71          512.28   
2 2021-10-28 21:10:11              26      261.09          339.42   
3 2021-09-02 04:37:16              21      214.01          278.21   
4 2023-03-17 16:47:17              42      206.46          268.40   

   day_of_week  is_working_day  is_holiday  month  hour  is_rush_hour  ...  \
0            3               1           0     12    12             0  ...   
1            3               1           0      1     8             1  ...   
2            3               1           0     10    21             0  ...   
3            3               1           0      9     4             0  ...   
4            4               1           0      3    16             0  ...   

   source_address_HSR Layout  source_address_Indiranagar  \
0                      False                       False   
1                      False                        True   
2                       True                       False   
3                       True                       False   
4                      False                        True   

   source_address_Koramangala  source_address_MG Road  \
0                        True                   False   
1                       False                   False   
2                       False                   False   
3                       False                   False   
4                       False                   False   

   source_address_Whitefield  destination_address_HSR Layout  \
0                      False                           False   
1                      False                           False   
2                      False                           False   
3                      False                           False   
4                      False                           False   

   destination_address_Indiranagar  destination_address_Koramangala  \
0                             True                            False   
1                            False                            False   
2                             True                            False   
3                             True                            False   
4                            False                            False   

   destination_address_MG Road  destination_address_Whitefield  
0                        False                           False  
1                         True                           False  
2                        False                           False  
3                        False                           False  
4                         True                           False  

[5 rows x 32 columns]

In [32]:
# Step 5: Splitting Data into Training and Testing Sets
X = data.drop(columns=['timestamp', 'number_of_cabs', 'adjusted_price'])  # 'timestamp' dropped if not useful
y_cabs = data['number_of_cabs']
y_price = data['adjusted_price']

In [33]:
X.head()

base_price  day_of_week  is_working_day  is_holiday  month  hour  \
0      256.97            3               1           0     12    12   
1      262.71            3               1           0      1     8   
2      261.09            3               1           0     10    21   
3      214.01            3               1           0      9     4   
4      206.46            4               1           0      3    16   

   is_rush_hour  is_weekend  weather_condition_Foggy  weather_condition_Rainy  \
0             0           0                    False                     True   
1             1           0                    False                     True   
2             0           0                    False                     True   
3             0           0                     True                    False   
4             0           0                    False                     True   

   ...  source_address_HSR Layout  source_address_Indiranagar  \
0  ...                      False                       False   
1  ...                      False                        True   
2  ...                       True                       False   
3  ...                       True                       False   
4  ...                      False                        True   

   source_address_Koramangala  source_address_MG Road  \
0                        True                   False   
1                       False                   False   
2                       False                   False   
3                       False                   False   
4                       False                   False   

   source_address_Whitefield  destination_address_HSR Layout  \
0                      False                           False   
1                      False                           False   
2                      False                           False   
3                      False                           False   
4                      False                           False   

   destination_address_Indiranagar  destination_address_Koramangala  \
0                             True                            False   
1                            False                            False   
2                             True                            False   
3                             True                            False   
4                            False                            False   

   destination_address_MG Road  destination_address_Whitefield  
0                        False                           False  
1                         True                           False  
2                        False                           False  
3                        False                           False  
4                         True                           False  

[5 rows x 29 columns]

In [76]:
X.columns

Index(['base_price', 'day_of_week', 'is_working_day', 'is_holiday', 'month',
       'hour', 'is_rush_hour', 'is_weekend', 'weather_condition_Foggy',
       'weather_condition_Rainy', 'weather_condition_Sunny',
       'weather_hour_interaction', 'lag_1', 'lag_2', 'rolling_mean_cabs',
       'lag_cabs_3', 'hour_weather_Rainy', 'hour_weather_Sunny',
       'hour_weather_Foggy', 'source_address_HSR Layout',
       'source_address_Indiranagar', 'source_address_Koramangala',
       'source_address_MG Road', 'source_address_Whitefield',
       'destination_address_HSR Layout', 'destination_address_Indiranagar',
       'destination_address_Koramangala', 'destination_address_MG Road',
       'destination_address_Whitefield'],
      dtype='object')

In [34]:
y_cabs.head()

0    34
1    24
2    26
3    21
4    42
Name: number_of_cabs, dtype: int64

In [35]:
y_price.head()

0    334.06
1    512.28
2    339.42
3    278.21
4    268.40
Name: adjusted_price, dtype: float64

In [38]:
from sklearn.model_selection import train_test_split

# Split data for cab availability prediction
X_train, X_test, y_train_cabs, y_test_cabs = train_test_split(X, y_cabs, test_size=0.2, random_state=42)

# Split data for cost prediction
y_train_price, y_test_price = y_price.loc[y_train_cabs.index], y_price.loc[y_test_cabs.index]

# 2. MODEL BUILDING :

## A) For Random Forest:

In [41]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [42]:
# Initialize and train models
rf_cabs = RandomForestRegressor()
rf_price = RandomForestRegressor()
gb_cabs = GradientBoostingRegressor()
gb_price = GradientBoostingRegressor()

In [43]:
rf_cabs.fit(X_train, y_train_cabs)
rf_price.fit(X_train, y_train_price)
gb_cabs.fit(X_train, y_train_cabs)
gb_price.fit(X_train, y_train_price)

GradientBoostingRegressor()

In [44]:
# Predictions
y_pred_rf_cabs = rf_cabs.predict(X_test)
y_pred_rf_price = rf_price.predict(X_test)
y_pred_gb_cabs = gb_cabs.predict(X_test)
y_pred_gb_price = gb_price.predict(X_test)


In [45]:
# Evaluation
print("Random Forest - Number of Cabs:")
print("MSE:", mean_squared_error(y_test_cabs, y_pred_rf_cabs))
print("R^2:", r2_score(y_test_cabs, y_pred_rf_cabs))

Random Forest - Number of Cabs:
MSE: 35.823741880341885
R^2: 0.8203992589505327


In [46]:
print("\nRandom Forest - Cost Prediction:")
print("MSE:", mean_squared_error(y_test_price, y_pred_rf_price))
print("R^2:", r2_score(y_test_price, y_pred_rf_price))


Random Forest - Cost Prediction:
MSE: 515.4250036922216
R^2: 0.9466111690394757


## B) Gradient Boosting :

In [47]:
print("\nGradient Boosting - Number of Cabs:")
print("MSE:", mean_squared_error(y_test_cabs, y_pred_gb_cabs))
print("R^2:", r2_score(y_test_cabs, y_pred_gb_cabs))


Gradient Boosting - Number of Cabs:
MSE: 14.742961773328043
R^2: 0.9260868150346215


In [48]:
print("\nGradient Boosting - Cost Prediction:")
print("MSE:", mean_squared_error(y_test_price, y_pred_gb_price))
print("R^2:", r2_score(y_test_price, y_pred_gb_price))


Gradient Boosting - Cost Prediction:
MSE: 136.32441265674424
R^2: 0.9858792239976978


# 3. VERIFICATION & VALIDATION :

## A) RandomForest :

In [63]:
# Evaluate Random Forest on Test Set
y_pred_cabs_rf_test = rf_cabs.predict(X_test)
y_pred_price_rf_test = rf_price.predict(X_test)

In [65]:
#for cabs
rf_cabs_test_mse = mean_squared_error(y_test_cabs, y_pred_cabs_rf_test)
rf_cabs_test_r2 = r2_score(y_test_cabs, y_pred_cabs_rf_test)

In [66]:
#for price
rf_price_test_mse = mean_squared_error(y_test_price, y_pred_price_rf_test)
rf_price_test_r2 = r2_score(y_test_price, y_pred_price_rf_test)

In [67]:
print(f"Random Forest - Number of Cabs (Test): MSE: {rf_cabs_test_mse}, R^2: {rf_cabs_test_r2}")
print(f"Random Forest - Cost Prediction (Test): MSE: {rf_price_test_mse}, R^2: {rf_price_test_r2}")

Random Forest - Number of Cabs (Test): MSE: 35.823741880341885, R^2: 0.8203992589505327
Random Forest - Cost Prediction (Test): MSE: 515.4250036922216, R^2: 0.9466111690394757


## B) Gradient Boosting :

In [68]:
# Evaluate Gradient Boosting on Test Set
y_pred_cabs_gb_test = gb_cabs.predict(X_test)
y_pred_price_gb_test = gb_price.predict(X_test)

In [69]:
#for cabs
gb_cabs_test_mse = mean_squared_error(y_test_cabs, y_pred_cabs_gb_test)
gb_cabs_test_r2 = r2_score(y_test_cabs, y_pred_cabs_gb_test)

In [70]:
#for price
gb_price_test_mse = mean_squared_error(y_test_price, y_pred_price_gb_test)
gb_price_test_r2 = r2_score(y_test_price, y_pred_price_gb_test)

In [71]:
print(f"Gradient Boosting - Number of Cabs (Test): MSE: {gb_cabs_test_mse}, R^2: {gb_cabs_test_r2}")
print(f"Gradient Boosting - Cost Prediction (Test): MSE: {gb_price_test_mse}, R^2: {gb_price_test_r2}")

Gradient Boosting - Number of Cabs (Test): MSE: 14.742961773328043, R^2: 0.9260868150346215
Gradient Boosting - Cost Prediction (Test): MSE: 136.32441265674424, R^2: 0.9858792239976978


# 4. DEPLOYMENT :

## A) Saving The Models

In [72]:
import joblib

In [74]:
# Save Random Forest models
joblib.dump(rf_cabs, 'rf_cabs_model.pkl')
joblib.dump(rf_price, 'rf_price_model.pkl')

['rf_price_model.pkl']

In [75]:
# Save Gradient Boosting models
joblib.dump(gb_cabs, 'gb_cabs_model.pkl')
joblib.dump(gb_price, 'gb_price_model.pkl')

['gb_price_model.pkl']